In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold


In [ ]:
pip install auto-sklearn

In [1]:
from autosklearn.classification import AutoSklearnClassifier

In [3]:
df = pd.read_csv('/content/final_table.csv')
df = df.iloc[: , 1:]
print(df.shape)
df.head()

(1506, 11)


,Classification as urban/semi-urban/rural_one-hot,log_Population,water_source_category_one-hot,log_angle,#status_id_one-hot,log_days_passed,#adm2_one-hot,log_dist,#pay_one-hot,water_tech_category_one-hot,Water demand (lpcd)_one-hot
0,0,19.343908,0,9.966469,0,11.388017,0,9.976453,0,0,0
1,1,17.471318,1,9.966516,0,11.381002,1,9.976761,0,0,1
2,2,16.842817,0,9.966564,1,11.375583,2,9.976714,0,0,2
3,1,17.471318,0,9.966564,0,11.382084,1,9.976842,0,0,1
4,1,17.471318,0,9.966564,0,11.379920,1,9.976903,0,0,1


In [4]:
df.columns


Index(['Classification as urban/semi-urban/rural_one-hot', 'log_Population',
       'water_source_category_one-hot', 'log_angle', '#status_id_one-hot',
       'log_days_passed', '#adm2_one-hot', 'log_dist', '#pay_one-hot',
       'water_tech_category_one-hot', 'Water demand (lpcd)_one-hot'],
      dtype='object')

In [5]:
def calc_new_col(row):
   if row['Water demand (lpcd)_one-hot'] == 0:
        return 'high'
   elif row['Water demand (lpcd)_one-hot'] == 1:
        return 'mid'
   else:
        return 'low'
   

df["target_priority"] = df.apply(calc_new_col, axis=1)
df = df.drop(["Water demand (lpcd)_one-hot"], axis = 1)

In [6]:
#X is the features in this dataset
X = df.drop(["target_priority"], axis = 1)
#Y is the vector with our Target Variables
Y = df['target_priority']

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=111, train_size = .8)

In [21]:
from autosklearn.metrics import (accuracy,
                                 f1,
                                 roc_auc,
                                 precision,
                                 average_precision,
                                 recall,
                                 log_loss)
time = 120


model =  AutoSklearnClassifier(time_left_for_this_task= time,
                            max_models_on_disc=5,
                            scoring_functions=[roc_auc, accuracy, f1, precision, recall, log_loss])

model.fit(X_train, Y_train)

[WARNING] [2022-09-25 11:35:52,782:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.
[WARNING] [2022-09-25 11:36:00,853:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.
[WARNING] [2022-09-25 11:36:07,394:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.
[WARNING] [2022-09-25 11:36:14,485:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.
[WARNING] [2022-09-25 11:36:17,443:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.
[WARNING] [2022-09-25 11:36:25,378:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.
[WARNING] 

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      max_models_on_disc=5, per_run_time_limit=12,
                      scoring_functions=[roc_auc, accuracy, f1, precision,
                                         recall, log_loss],
                      time_left_for_this_task=120)

In [22]:
model.leaderboard(detailed = True)

,rank,ensemble_weight,type,cost,duration,config_id,train_loss,seed,start_time,end_time,budget,status,data_preprocessors,feature_preprocessors,balancing_strategy,config_origin
model_id,,,,,,,,,,,,,,,,
2,1,0.22,random_forest,0.0,2.037675,1,0.0,0,1.664106e+09,1.664106e+09,0.0,StatusType.SUCCESS,[],[no_preprocessing],none,Initial design
3,2,0.20,random_forest,0.0,2.016919,2,0.0,0,1.664106e+09,1.664106e+09,0.0,StatusType.SUCCESS,[],[select_percentile_classification],none,Initial design
4,3,0.14,liblinear_svc,0.0,1.105969,3,0.0,0,1.664106e+09,1.664106e+09,0.0,StatusType.SUCCESS,[],[polynomial],weighting,Initial design
5,4,0.36,gradient_boosting,0.0,1.349331,4,0.0,0,1.664106e+09,1.664106e+09,0.0,StatusType.SUCCESS,[],[no_preprocessing],weighting,Initial design
7,5,0.08,random_forest,0.0,2.172094,6,0.0,0,1.664106e+09,1.664106e+09,0.0,StatusType.SUCCESS,[],[liblinear_svc_preprocessor],weighting,Initial design


In [30]:
 model.show_models()


{2: {'model_id': 2,
  'rank': 1,
  'cost': 0.0,
  'ensemble_weight': 0.22,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x7fdf68dd1f50>,
  'balancing': Balancing(random_state=1),
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice at 0x7fdf686f5e90>,
  'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice at 0x7fdf686f58d0>,
  'sklearn_classifier': RandomForestClassifier(max_features=3, n_estimators=512, n_jobs=1,
                         random_state=1, warm_start=True)},
 3: {'model_id': 3,
  'rank': 2,
  'cost': 0.0,
  'ensemble_weight': 0.2,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x7fdf64b59290>,
  'balancing': Balancing(random_state=1),
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice at 0x7fdf68a293d0>,
  'classifier': <autoskle

In [31]:
import pickle
# save model
with open('water auto-classifier.pkl', 'wb') as f:
    pickle.dump(model, f)

In [32]:
# load model
with open('water auto-classifier.pkl', 'rb') as f:
    classifier = pickle.load(f)

In [41]:
X_test

,Classification as urban/semi-urban/rural_one-hot,log_Population,water_source_category_one-hot,log_angle,#status_id_one-hot,log_days_passed,#adm2_one-hot,log_dist,#pay_one-hot,water_tech_category_one-hot
993,1,17.881287,0,9.966400,1,11.375039,7,9.976049,0,0
635,2,18.257337,0,9.966504,1,11.388017,10,9.976560,0,0
1212,2,16.842817,0,9.966537,1,11.376668,2,9.976520,0,0
320,0,20.055109,0,9.966453,0,11.377753,3,9.976301,0,0
328,1,17.471318,1,9.966564,0,11.381002,1,9.976929,0,0
...,...,...,...,...,...,...,...,...,...,...
1366,2,16.842817,0,9.966564,0,11.377753,2,9.976592,0,0
900,0,18.278577,1,9.966478,0,11.390169,8,9.976319,1,0
1235,1,19.031365,0,9.966511,0,11.379920,4,9.976716,0,0
306,0,18.278577,0,9.966479,0,11.390706,8,9.976299,0,0


In [37]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
prediction = classifier.predict(X_test)

print(accuracy_score(Y_test, prediction))
print(f1_score(Y_test, prediction, average='micro'))
print(recall_score(Y_test, prediction, average='micro'))
print(precision_score(Y_test, prediction, average='micro'))

1.0
1.0
1.0
1.0


Resuing model with original data structure


In [89]:
#to reuse
import numpy as np
import json

with open('/content/one_hot_cols.json') as json_file:
   encoded_data = json.load(json_file)

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def lg(x):
  return np.log2(x)

expected_columns = ['Classification as urban/semi-urban/rural_one-hot', 'log_Population',
       'water_source_category_one-hot', 'log_angle', '#status_id_one-hot',
       'log_days_passed', '#adm2_one-hot', 'log_dist', '#pay_one-hot',
       'water_tech_category_one-hot', 'Water demand (lpcd)_one-hot']
      
def f_engineer():
  inp = []
  print( "Enter classification as of the area as \n 0. urban \n 1. semi-urban \n 2. rural: \n (Enter 0, 1, 2)")
  a1 = int(input())
  inp.append(a1)

  print( "Enter the population of the area: (Enter a positive integer)")
  a2 = int(input())
  inp.append(lg(a2)) #log population

  print( "Enter the water source category of the area as \n 0. well \n 1. spring \n (Enter 0, 1)")
  a3 = int(input())
  inp.append(a3)

  print( "Enter the latitude of the area:")
  a4 = int(input())
  print( "Enter the latitude of the area:")
  a5 = int(input())
  a4, a5 = cart2pol(a4, a5)

  inp.append(lg(a4 + 1000)) #log angle

  print( "Enter the status id of water source as \n 0. Yes (Functional) \n 1. No (non-functional): \n (Enter 0, 1)")
  a6 = int(input())
  inp.append(a6)

  print( "Enter the number of days passed since installation of water source: (Enter a postive integer)")
  a7 = int(input())
  inp.append(lg(a7))

  print( "Enter the local government name as")
  i = 0
  for adm in encoded_data['#adm2']:
    print( str(i) + ". " + str(adm) )
    i = i + 1
  print('(Enter a number from 0 to ' + str(i) + ')')
  print( str(i) + ". " + 'Unknown' )
  a8 = int(input())
  inp.append(a8)

  inp.append(lg(a5 +1000)) #log of distance

  print( "Enter the payment method as")
  i = 0
  for adm in encoded_data['#pay']:
    print( str(i) + ". " + str(adm) )
    i = i + 1
  print( str(i) + ". " + 'Unknown' )
  print('(Enter a number from 0 to ' + str(i) + ')')
  a9 = int(input())
  inp.append(a9)

  print( "Enter the water source category as")
  i = 0
  for adm in encoded_data['water_tech_category']:
    print( str(i) + ". " + str(adm) )
    i = i + 1
  print( str(i) + ". " + 'Unknown' )
  print('(Enter a number from 0 to ' + str(i) + ')')
  a10 = int(input())
  inp.append(a10)

  return inp



In [82]:
# load model
with open('water auto-classifier.pkl', 'rb') as f:
    classifier = pickle.load(f)

#input_data = f_engineer()

def make_prediction(input_data):

  column_names =['Classification as urban/semi-urban/rural_one-hot', 'log_Population',
        'water_source_category_one-hot', 'log_angle', '#status_id_one-hot',
        'log_days_passed', '#adm2_one-hot', 'log_dist', '#pay_one-hot',
        'water_tech_category_one-hot']


  data = input_data
  df = pd.DataFrame([data], columns = column_names)

  make_prediction = classifier.predict(df)
  print("The priority of target area is classified as: ")
  return  make_prediction

In [90]:
input_data = f_engineer()
x= make_prediction(input_data)
print(x[0])

Enter classification as of the area as 
 0. urban 
 1. semi-urban 
 2. rural: 
 (Enter 0, 1, 2)
0
Enter the population of the area: (Enter a positive integer)
4356
Enter the water source category of the area as 
 0. well 
 1. spring 
 (Enter 0, 1)
1
Enter the latitude of the area:
89
Enter the latitude of the area:
87
Enter the status id of water source as 
 0. Yes (Functional) 
 1. No (non-functional): 
 (Enter 0, 1)
1
Enter the number of days passed since installation of water source: (Enter a postive integer)
89
Enter the local government name as
0. Kosofe
1. Epe
2. Ibeju-Lekki
3. Alimosho
4. Ikorodu
5. Agege
6. Ojo
7. Badagry
8. Lagos Mainland
9. Mushin
10. Ikeja
11. Lagos Island
12. Shomolu
13. Eti-Osa
14. Ifako-Ijaye
15. Ajeromi-Ifelodun
16. Surulere
17. Oshodi
18. Amuwo-Odofin
(Enter a number from 0 to 19)
19. Unknown
14
Enter the payment method as
0. No
1. Yes Point of collection
2. Yes At breakdown
3. Yes Periodic Levy
4. Unknown
(Enter a number from 0 to 4)
0
Enter the water 